In [15]:
import networkx as nx
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict


class MaxConcurrentFlowSolver:
    """
    Finds the maximum flow that all sessions can achieve simultaneously.

    Solving Fractional Multi Commodity Flow is Polynomial Time.
    However, the print_solution method itself is NP cuz simple path finding.
    
    Solving Integral Multi Commodity Flow is NP hard.
    """
    
    def __init__(self, G: nx.Graph, sessions: List[Tuple[int, int]]):
        """
        Initialize the solver with a network and sessions.
        
        Args:
            G: NetworkX undirected graph with edge capacities stored as 'capacity' attribute
            sessions: List of (source, target) tuples representing commodity flows
        """
        self.G = G.copy()
        self.sessions = sessions
        self.num_nodes = G.number_of_nodes()
        self.num_edges = G.number_of_edges()
        self.num_commodities = len(sessions)
        
        # Store edges in a consistent order
        self.edges = list(G.edges())
        self.edge_index = {e: i for i, e in enumerate(self.edges)}
        
        # Add reverse edges to the edge index for undirected graph
        for (u, v) in list(self.edge_index.keys()):
            self.edge_index[(v, u)] = self.edge_index[(u, v)]
        
        # Extract edge capacities
        self.capacities = np.array([G[u][v].get('capacity', float('inf')) for u, v in self.edges])
        
        # Create a directed graph for tracking flows (to handle edge directions properly)
        self.flow_graph = nx.DiGraph()
        for u, v in self.edges:
            capacity = G[u][v].get('capacity', float('inf'))
            self.flow_graph.add_edge(u, v, capacity=capacity)
            self.flow_graph.add_edge(v, u, capacity=capacity)
        
        # Initialize results
        self.flows = None
        self.max_concurrent_flow = None
        self.status = None
    
    def solve(self):
        """
        Solve the maximum concurrent flow problem using linear programming.
        
        Returns:
            Dict containing solution status and optimal flows
        """
        # Create flow variables for each commodity on each edge
        # f[k][i][j] represents flow of commodity k on edge (i,j)
        f = {}
        for k in range(self.num_commodities):
            f[k] = {}
            for i in range(self.num_nodes):
                f[k][i] = {}
                for j in self.G.neighbors(i):
                    f[k][i][j] = cp.Variable(nonneg=True)
        
        # Create variable for the maximum concurrent flow
        # This represents how much flow each session can carry
        mcf = cp.Variable(nonneg=True)
        
        # Objective: Maximize the concurrent flow
        objective = cp.Maximize(mcf)
        
        # Constraints
        constraints = []
        
        # 1. Capacity constraints for each edge (sum of all commodity flows <= edge capacity)
        for u, v in self.edges:
            edge_flow_sum = 0
            for k in range(self.num_commodities):
                edge_flow_sum += f[k][u][v] + f[k][v][u]  # Sum both directions for undirected
            constraints.append(edge_flow_sum <= self.G[u][v].get('capacity', float('inf')))
        
        # 2. Flow conservation constraints
        for k, (source, target) in enumerate(self.sessions):
            for i in range(self.num_nodes):
                if i != source and i != target:  # For intermediate nodes
                    # Sum of incoming flows equals sum of outgoing flows
                    flow_balance = 0
                    for j in self.G.neighbors(i):
                        flow_balance += f[k][j][i] - f[k][i][j]
                    constraints.append(flow_balance == 0)
        
        # 3. Flow requirements - each session must achieve the mcf value
        for k, (source, target) in enumerate(self.sessions):
            # Calculate the net outflow at source
            source_outflow = 0
            for j in self.G.neighbors(source):
                source_outflow += f[k][source][j] - f[k][j][source]
            constraints.append(source_outflow == mcf)
            
            # The net inflow at target should equal the outflow at source
            target_inflow = 0
            for i in self.G.neighbors(target):
                target_inflow += f[k][i][target] - f[k][target][i]
            constraints.append(target_inflow == mcf)
        
        # Solve the problem
        problem = cp.Problem(objective, constraints)
        problem.solve(solver=cp.ECOS)
        
        # Store results
        self.status = problem.status
        self.max_concurrent_flow = mcf.value
        
        # Extract flow values for each commodity on each edge
        self.flows = {}
        for k in range(self.num_commodities):
            self.flows[k] = {}
            for u in range(self.num_nodes):
                for v in self.G.neighbors(u):
                    if (u, v) not in self.flows[k]:
                        # Get the flow values in both directions
                        forward_flow = f[k][u][v].value
                        backward_flow = f[k][v][u].value if v in f[k] and u in f[k][v] else 0
                        
                        # Calculate effective flow (can't have flow in both directions simultaneously)
                        if forward_flow > backward_flow:
                            self.flows[k][(u, v)] = forward_flow - backward_flow
                            self.flows[k][(v, u)] = 0
                        else:
                            self.flows[k][(v, u)] = backward_flow - forward_flow
                            self.flows[k][(u, v)] = 0
        
        return {
            'status': self.status,
            'max_concurrent_flow': self.max_concurrent_flow,
            'flows': self.flows
        }
    
    def get_flow_dict(self) -> Dict:
        """
        Returns a dictionary of flows for each session and edge.
        
        Returns:
            Dict: {session_idx: {(u, v): flow_value}}
        """
        if self.flows is None:
            raise ValueError("Problem must be solved before getting flows")
        return self.flows
    
    def visualize_network(self, figsize=(12, 8)):
        """
        Visualize the network with edge capacities and optimal flows.
        """
        if self.flows is None:
            raise ValueError("Problem must be solved before visualization")
        
        plt.figure(figsize=figsize)
        
        # Create position layout
        pos = nx.spring_layout(self.G, seed=42)
        
        # Draw nodes
        nx.draw_networkx_nodes(self.G, pos, node_size=500)
        
        # Draw edges with capacity labels
        edge_labels = {(u, v): f"cap: {self.G[u][v].get('capacity', '∞')}" for u, v in self.edges}
        nx.draw_networkx_edges(self.G, pos, width=1.0, alpha=0.5)
        nx.draw_networkx_edge_labels(self.G, pos, edge_labels=edge_labels)
        
        # Draw node labels
        nx.draw_networkx_labels(self.G, pos)
        
        # Add source and target information
        for k, (source, target) in enumerate(self.sessions):
            plt.annotate(f"Session {k}: {source}→{target} (flow: {self.max_concurrent_flow:.4f})",
                        xy=(0, 0), xytext=(0, -30 - 10 * k),
                        xycoords=('axes fraction'), textcoords='offset points',
                        ha='left', va='top')
        
        plt.title(f"Maximum Concurrent Flow: {self.max_concurrent_flow:.4f}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    def print_solution(self):
        """
        Print the solution details.
        """
        if self.flows is None:
            raise ValueError("Problem must be solved before printing solution")
        
        print(f"Solution status: {self.status}")
        print(f"Maximum concurrent flow: {self.max_concurrent_flow:.4f}")
        
        for k, (source, target) in enumerate(self.sessions):
            print(f"\nSession {k}: {source} → {target} (flow: {self.max_concurrent_flow:.4f})")
            
            # Create a directed flow network for this commodity
            flow_net = nx.DiGraph()
            for u, v in self.G.edges():
                flow_uv = self.flows[k].get((u, v), 0)
                flow_vu = self.flows[k].get((v, u), 0)
                
                if flow_uv > 1e-6:
                    flow_net.add_edge(u, v, flow=flow_uv)
                if flow_vu > 1e-6:
                    flow_net.add_edge(v, u, flow=flow_vu)
            
            # Find all simple paths from source to target in the flow network
            try:
                all_paths = list(nx.all_simple_paths(flow_net, source, target))
                
                # Calculate flow for each path
                path_flows = []
                remaining_flow = self.max_concurrent_flow
                
                for path in all_paths:
                    if remaining_flow < 1e-6:
                        break
                        
                    # Find the minimum flow on the path
                    min_flow = float('inf')
                    for i in range(len(path) - 1):
                        u, v = path[i], path[i + 1]
                        edge_flow = flow_net[u][v].get('flow', 0)
                        min_flow = min(min_flow, edge_flow)
                    
                    # Skip paths with no flow
                    if min_flow < 1e-6:
                        continue
                    
                    # Adjust for remaining flow
                    path_flow = min(min_flow, remaining_flow)
                    path_flows.append((path, path_flow))
                    remaining_flow -= path_flow
                    
                    # Reduce the flow on this path
                    for i in range(len(path) - 1):
                        u, v = path[i], path[i + 1]
                        flow_net[u][v]['flow'] -= path_flow
                
                # Print paths
                total_session_flow = 0
                for path, flow in path_flows:
                    print(f"  Path: {' → '.join(map(str, path))}, Flow: {flow:.4f}")
                    total_session_flow += flow
                
                print(f"  Total session flow: {total_session_flow:.4f}")
                
            except nx.NetworkXNoPath:
                print(f"  No flow path found from {source} to {target}")
                print(f"  Total session flow: 0.0000")
    
    def visualize_flows(self, figsize=(15, 10)):
        """
        Visualize the network with flows for each session.
        """
        if self.flows is None:
            raise ValueError("Problem must be solved before visualization")
        
        # Create a separate visualization for each session
        for k, (source, target) in enumerate(self.sessions):
            plt.figure(figsize=figsize)
            pos = nx.spring_layout(self.G, seed=42)
            
            # Create a directed graph for this commodity's flow
            flow_graph = nx.DiGraph()
            flow_graph.add_nodes_from(self.G.nodes())
            
            # Add edges with flow values
            edge_colors = []
            edge_widths = []
            
            for u, v in self.G.edges():
                flow_uv = self.flows[k].get((u, v), 0)
                flow_vu = self.flows[k].get((v, u), 0)
                
                if flow_uv > 1e-6:
                    flow_graph.add_edge(u, v, weight=flow_uv)
                    edge_colors.append(flow_uv)
                    edge_widths.append(1 + 5 * flow_uv / self.max_concurrent_flow if self.max_concurrent_flow > 0 else 1)
                
                if flow_vu > 1e-6:
                    flow_graph.add_edge(v, u, weight=flow_vu)
                    edge_colors.append(flow_vu)
                    edge_widths.append(1 + 5 * flow_vu / self.max_concurrent_flow if self.max_concurrent_flow > 0 else 1)
            
            # Draw the nodes
            nx.draw_networkx_nodes(flow_graph, pos, node_size=700,
                                  node_color=['red' if n == source else 'green' if n == target else 'lightblue' 
                                             for n in flow_graph.nodes()])
            
            # Draw the edges with flow
            if flow_graph.edges():
                edges = nx.draw_networkx_edges(flow_graph, pos, width=edge_widths, edge_color=edge_colors, 
                                             edge_cmap=plt.cm.Blues, edge_vmin=0, edge_vmax=self.max_concurrent_flow,
                                             connectionstyle='arc3,rad=0.1')  # Curved edges for directed
                
                # Add a colorbar
                sm = plt.cm.ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(0, self.max_concurrent_flow))
                sm.set_array([])
                cbar = plt.colorbar(sm)
                cbar.set_label('Flow Amount')
                
                # Add edge labels with flow values
                edge_labels = {(u, v): f"{flow_graph[u][v]['weight']:.2f}" for u, v in flow_graph.edges()}
                nx.draw_networkx_edge_labels(flow_graph, pos, edge_labels=edge_labels, label_pos=0.3)
            
            # Draw node labels
            nx.draw_networkx_labels(flow_graph, pos)
            
            # Set title
            plt.title(f"Session {k}: {source} → {target} (Flow: {self.max_concurrent_flow:.4f})")
            plt.axis('off')
            plt.tight_layout()
            plt.show()


# Example usage
def run_example():
    # Create a sample network
    G = nx.Graph()
    
    # Add nodes and edges with capacities
    edges = [
        (0,3),
        (1,3),
        (2,3),
        (0,4),
        (1,4),
        (2,4)
    ]
    for u, v in edges:
        G.add_edge(u, v, capacity=1)
    
    # Define sessions: (source, target)
    sessions = [
        (0, 1),
        (1, 2),
        (0, 2),
        (3, 4)
    ]
    
    # Create and solve the maximum concurrent flow problem
    solver = MaxConcurrentFlowSolver(G, sessions)
    result = solver.solve()
    
    # Display results
    solver.print_solution()
    # solver.visualize_network()
    # solver.visualize_flows()


if __name__ == "__main__":
    run_example()

Solution status: optimal
Maximum concurrent flow: 0.7500

Session 0: 0 → 1 (flow: 0.7500)
  Path: 0 → 3 → 1, Flow: 0.3750
  Path: 0 → 4 → 1, Flow: 0.3750
  Total session flow: 0.7500

Session 1: 1 → 2 (flow: 0.7500)
  Path: 1 → 3 → 2, Flow: 0.3750
  Path: 1 → 4 → 2, Flow: 0.3750
  Total session flow: 0.7500

Session 2: 0 → 2 (flow: 0.7500)
  Path: 0 → 3 → 2, Flow: 0.3750
  Path: 0 → 4 → 2, Flow: 0.3750
  Total session flow: 0.7500

Session 3: 3 → 4 (flow: 0.7500)
  Path: 3 → 0 → 4, Flow: 0.2500
  Path: 3 → 1 → 4, Flow: 0.2500
  Path: 3 → 2 → 4, Flow: 0.2500
  Total session flow: 0.7500


In [25]:
import networkx as nx
import itertools
from pulp import *
import matplotlib.pyplot as plt

class EntropyCalculusAnalyzer:
    """
    A general script for analyzing network capacity using entropy calculus.
    """
    def __init__(self):
        self.G = None  # Undirected graph
        self.DG = None  # Directed graph
        self.source_sink_pairs = []
        self.entropy_vars = {}  # Dictionary to store all entropy variables
        self.results = {}
        
    def create_network(self, edges, capacities=None):
        """
        Create a network from a list of edges.
        
        Args:
            edges: List of (u, v) tuples representing edges
            capacities: Dictionary mapping edges to capacities, defaults to 1
        """
        self.G = nx.Graph()
        self.G.add_edges_from(edges)
        
        # Set capacities
        if capacities:
            for edge, capacity in capacities.items():
                u, v = edge
                self.G[u][v]['capacity'] = capacity
        else:
            # Default capacity is 1
            nx.set_edge_attributes(self.G, 1, 'capacity')
            
        # Create directed version
        self.DG = self._derive_directed_graph()
        
        return self
    
    def set_source_sink_pairs(self, pairs):
        """
        Set the source-sink pairs for the network.
        
        Args:
            pairs: List of (source, sink) tuples
        """
        self.source_sink_pairs = pairs
        return self
    
    def _derive_directed_graph(self):
        """
        Derive a directed graph by replacing each undirected edge
        with two directed edges.
        """
        DG = nx.DiGraph()
        
        # Add all nodes
        DG.add_nodes_from(self.G.nodes)
        
        # Replace each undirected edge with two directed edges
        for u, v, data in self.G.edges(data=True):
            capacity = data.get('capacity', 1)
            DG.add_edge(u, v, capacity=capacity)
            DG.add_edge(v, u, capacity=capacity)
        
        return DG
    
    def get_all_cuts(self):
        """
        Generate all possible cuts of the graph.
        A cut is a partition of nodes into two sets.
        """
        nodes = list(self.G.nodes)
        cuts = []
        
        # Generate all possible subsets of nodes
        for i in range(1, len(nodes)):
            for subset in itertools.combinations(nodes, i):
                S = set(subset)
                S_complement = set(nodes) - S
                cuts.append((S, S_complement))
        
        return cuts
    
    def get_incoming_edges(self, S):
        """Get incoming edges to a set of nodes in the directed graph."""
        incoming = []
        for v in S:
            for u in self.DG.predecessors(v):
                if u not in S:
                    incoming.append((u, v))
        return incoming
    
    def get_outgoing_edges(self, S):
        """Get outgoing edges from a set of nodes in the directed graph."""
        outgoing = []
        for u in S:
            for v in self.DG.successors(u):
                if v not in S:
                    outgoing.append((u, v))
        return outgoing
    
    def formulate_lp(self):
        """
        Formulate a linear program for network capacity analysis.
        """
        # Create LP problem
        prob = LpProblem("Network_Capacity", LpMaximize)
        
        # 1. Create variables for all sessions (source-sink pairs)
        for i, (source, sink) in enumerate(self.source_sink_pairs):
            var_name = f"X{i+1}"
            self.entropy_vars[var_name] = LpVariable(var_name, lowBound=0)
        
        # 2. Create variables for all directed edges
        for u, v in self.DG.edges():
            var_name = f"H_{u}_{v}"
            self.entropy_vars[var_name] = LpVariable(var_name, lowBound=0)
        
        # 3. Create variables for all subsets used in input-output inequalities
        all_cuts = self.get_all_cuts()
        for S, S_complement in all_cuts:
            s_label = '_'.join(sorted(S))
            # Check if any inputs (edges or sources) or outputs (edges or sinks) exist for S
            has_inputs = bool(self.get_incoming_edges(S) or any(s in S for s,t in self.source_sink_pairs))
            has_outputs = bool(self.get_outgoing_edges(S) or any(t in S for s,t in self.source_sink_pairs))

            if has_inputs:
                self.entropy_vars[f"H_in_{s_label}"] = LpVariable(f"H_in_{s_label}", lowBound=0)
            if has_outputs:
                 self.entropy_vars[f"H_out_{s_label}"] = LpVariable(f"H_out_{s_label}", lowBound=0)
            # Need H_in_out only if both inputs and outputs exist conceptually for IO inequality
            if has_inputs and has_outputs:
                 self.entropy_vars[f"H_in_out_{s_label}"] = LpVariable(f"H_in_out_{s_label}", lowBound=0)

        
        # 4. Set objective: maximize H(X1)
        prob += self.entropy_vars["X1"]
        
        # 5. Add constraint: all sessions have the same entropy
        for i in range(2, len(self.source_sink_pairs) + 1):
            prob += self.entropy_vars[f"X{i}"] == self.entropy_vars["X1"]
        
        # 6. Add capacity constraints
        for u, v in self.G.edges():
            prob += (self.entropy_vars[f"H_{u}_{v}"] + self.entropy_vars[f"H_{v}_{u}"] 
                    <= self.G[u][v].get('capacity', 1))
        
        # 7. Add input-output inequalities for all cuts
        for S, S_complement in all_cuts:
            s_label = '_'.join(sorted(S))
            
            # Identify components for this cut
            in_edges = self.get_incoming_edges(S)
            out_edges = self.get_outgoing_edges(S)
            
            sources_in_S_idx = [i for i, (s, t) in enumerate(self.source_sink_pairs) if s in S]
            sinks_in_S_idx = [i for i, (s, t) in enumerate(self.source_sink_pairs) if t in S]

            sources_in_S_vars = [self.entropy_vars[f"X{i+1}"] for i in sources_in_S_idx]
            sinks_in_S_vars = [self.entropy_vars[f"X{i+1}"] for i in sinks_in_S_idx] # Used for H_out and H_in_out bounds

            in_edge_vars = [self.entropy_vars[f"H_{u}_{v}"] for u, v in in_edges]
            out_edge_vars = [self.entropy_vars[f"H_{u}_{v}"] for u, v in out_edges]

            # Define H_in_S and its relations (if it exists)
            in_S_name = f"H_in_{s_label}"
            if in_S_name in self.entropy_vars:
                H_in_S = self.entropy_vars[in_S_name]
                # Upper bound relaxation: H_in_S <= sum of components
                prob += H_in_S <= lpSum(in_edge_vars) + lpSum(sources_in_S_vars), f"UpperBound_{in_S_name}"
                # Monotonicity: Each component <= H_in_S
                for edge_var in in_edge_vars:
                    prob += edge_var <= H_in_S, f"MonoInEdge_{edge_var.name}_in_{s_label}"
                for source_var in sources_in_S_vars:
                    prob += source_var <= H_in_S, f"MonoInSource_{source_var.name}_in_{s_label}"

            # Define H_out_S and its relations (if it exists)
            out_S_name = f"H_out_{s_label}"
            if out_S_name in self.entropy_vars:
                H_out_S = self.entropy_vars[out_S_name]
                 # Upper bound relaxation: H_out_S <= sum of components (edges out + sinks in S)
                prob += H_out_S <= lpSum(out_edge_vars) + lpSum(sinks_in_S_vars), f"UpperBound_{out_S_name}"
                 # Monotonicity: Each component <= H_out_S
                for edge_var in out_edge_vars:
                     prob += edge_var <= H_out_S, f"MonoOutEdge_{edge_var.name}_out_{s_label}"
                for sink_var in sinks_in_S_vars: # X_i where t_i in S
                     prob += sink_var <= H_out_S, f"MonoOutSink_{sink_var.name}_out_{s_label}"


            # Define H_in_out_S and apply IO (if it exists)
            both_S_name = f"H_in_out_{s_label}"
            if both_S_name in self.entropy_vars:
                H_in_out_S = self.entropy_vars[both_S_name]
                
                # Upper bound relaxation: H_in_out_S <= sum of all components (in/out edges + sources/sinks in S)
                prob += H_in_out_S <= lpSum(in_edge_vars) + lpSum(out_edge_vars) + \
                                      lpSum(sources_in_S_vars) + lpSum(sinks_in_S_vars), f"UpperBound_{both_S_name}"

                # Monotonicity H_in <= H_in_out and H_out <= H_in_out
                if in_S_name in self.entropy_vars:
                    prob += self.entropy_vars[in_S_name] <= H_in_out_S, f"Mono_In_vs_InOut_{s_label}"
                if out_S_name in self.entropy_vars:
                     prob += self.entropy_vars[out_S_name] <= H_in_out_S, f"Mono_Out_vs_InOut_{s_label}"

                # Apply Input-Output Inequality Relaxation: H(in U out) <= H(in)
                # Use the variables representing the upper bounds
                if in_S_name in self.entropy_vars: # Check H_in_S exists
                    prob += H_in_out_S <= self.entropy_vars[in_S_name], f"IO_{s_label}"


        
        # 8. Add submodularity constraints
        # For each pair of variables S1, S2 where S1 is a subset of S2
        # we add H(S1) <= H(S2)

        # 9. Add joint decodability constraint per sink
        from collections import defaultdict
        
        # Group session variables by sink node
        sink_to_sessions = defaultdict(list)
        for i, (source, sink) in enumerate(self.source_sink_pairs):
            var_name = f"X{i+1}"
            sink_to_sessions[sink].append(var_name)
        
        # For each sink with one or more session targeting it
        for sink, session_vars in sink_to_sessions.items():
            in_edges = list(self.DG.in_edges(sink))
            if in_edges:
                lhs = lpSum(self.entropy_vars[x] for x in session_vars)
                rhs = lpSum(self.entropy_vars[f"H_{u}_{v}"] for u, v in in_edges)
                print(lhs,rhs)
                prob += lhs <= rhs, f"Joint_Decodability_at_{sink}"


        
        # For session variables in relation to edge variables
        for i, (source, sink) in enumerate(self.source_sink_pairs):
            var_name = f"X{i+1}"
            
            # Check which cuts have this session's source and sink separated
            for S, S_complement in self.get_all_cuts():
                if ((source in S and sink in S_complement) or 
                    (source in S_complement and sink in S)):
                    
                    # Session entropy should be <= entropy of edges crossing the cut
                    in_edges = self.get_incoming_edges(S)
                    out_edges = self.get_outgoing_edges(S)
                    
                    if in_edges and out_edges:
                        both_S_name = f"H_in_out_{'_'.join(sorted(S))}"
                        prob += self.entropy_vars[var_name] <= self.entropy_vars[both_S_name]
        
        return prob
    
    def analyze_network(self):
        """
        Analyze the network by formulating and solving the LP.
        """
        # Formulate LP
        prob = self.formulate_lp()
        
        # Solve with CBC solver
        prob.solve(PULP_CBC_CMD(msg=False))
        
        # Store results
        self.results['status'] = LpStatus[prob.status]
        
        if prob.status == LpStatusOptimal:
            self.results['max_rate'] = value(prob.objective)
            
            # Store variable values
            var_values = {}
            for name, var in self.entropy_vars.items():
                var_values[name] = var.value()
            self.results['variables'] = var_values
        
        return self.results
    

    
    def print_results(self):
        """Print the analysis results."""
        print("\n====== Network Capacity Analysis Results ======")
        print(f"Network: {len(self.G.nodes)} nodes, {len(self.G.edges())} edges")
        print(f"Source-sink pairs: {self.source_sink_pairs}")
        
        if 'max_rate' in self.results:
            print(f"\nMaximum achievable rate: {self.results['max_rate']}")
            
            if 'variables' in self.results:
                print("\nKey Variable Values:")
                for i in range(len(self.source_sink_pairs)):
                    print(f"H(X{i+1}) = {self.results['variables'][f'X{i+1}']}")
                    
                # Print a few edge entropy values as examples
                count = 0
                for edge, value in self.results['variables'].items():
                    if edge.startswith("H_") and "_" in edge and count < 100:
                        print(f"{edge} = {value}")
                        count += 1
        
        print("\n===========================================")


# Example Usage
if __name__ == "__main__":
    # Create analyzer
    analyzer = EntropyCalculusAnalyzer()
    
    # Example: Create K3,2 network
    edges = [
        ('a', 'd'), ('a', 'e'),
        ('b', 'd'), ('b', 'e'),
        ('c', 'd'), ('c', 'e')
    ]
    analyzer.create_network(edges)
    
    # Set source-sink pairs (cyclic configuration for K3,2)
    pairs = [
        ('a', 'b'),  # X1
        ('b', 'c'),  # X2
        ('a', 'c'),  # X3
        ('d', 'e')   # X4
    ]
    analyzer.set_source_sink_pairs(pairs)
    

    
    # Analyze network
    analyzer.analyze_network()
    analyzer.print_results()
    


X1 H_d_b + H_e_b
X2 + X3 H_d_c + H_e_c
X4 H_a_e + H_b_e + H_c_e

====== Network Capacity Analysis Results ======
Network: 5 nodes, 6 edges
Source-sink pairs: [('a', 'b'), ('b', 'c'), ('a', 'c'), ('d', 'e')]

Maximum achievable rate: 1.0

Key Variable Values:
H(X1) = 1.0
H(X2) = 1.0
H(X3) = 1.0
H(X4) = 1.0
H_a_d = 0.0
H_a_e = 1.0
H_d_a = 0.0
H_d_b = 1.0
H_d_c = 1.0
H_e_a = 0.0
H_e_b = 0.0
H_e_c = 1.0
H_b_d = 0.0
H_b_e = 0.0
H_c_d = 0.0
H_c_e = 0.0
H_in_a = 1.0
H_out_a = 1.0
H_in_out_a = 1.0
H_in_d = 1.0
H_out_d = 1.0
H_in_out_d = 1.0
H_in_e = 1.0
H_out_e = 1.0
H_in_out_e = 1.0
H_in_b = 1.0
H_out_b = 1.0
H_in_out_b = 1.0
H_in_c = 1.0
H_out_c = 1.0
H_in_out_c = 1.0
H_in_a_d = 1.0
H_out_a_d = 1.0
H_in_out_a_d = 1.0
H_in_a_e = 1.0
H_out_a_e = 1.0
H_in_out_a_e = 1.0
H_in_a_b = 1.0
H_out_a_b = 1.0
H_in_out_a_b = 1.0
H_in_a_c = 1.0
H_out_a_c = 1.0
H_in_out_a_c = 1.0
H_in_d_e = 1.0
H_out_d_e = 1.0
H_in_out_d_e = 1.0
H_in_b_d = 1.0
H_out_b_d = 1.0
H_in_out_b_d = 1.0
H_in_c_d = 1.0
H_out_c_d = 1.

In [23]:
import networkx as nx
import itertools
from pulp import *
import matplotlib.pyplot as plt
from collections import defaultdict # Added import

class EntropyCalculusAnalyzer:
    """
    A general script for analyzing network capacity using entropy calculus.
    """
    def __init__(self):
        self.G = None  # Undirected graph
        self.DG = None  # Directed graph
        self.source_sink_pairs = []
        self.entropy_vars = {}  # Dictionary to store all entropy variables
        self.results = {}

    def create_network(self, edges, capacities=None):
        self.G = nx.Graph()
        self.G.add_edges_from(edges)
        if capacities:
            for edge, capacity in capacities.items():
                u, v = edge
                self.G[u][v]['capacity'] = capacity
        else:
            nx.set_edge_attributes(self.G, 1, 'capacity')
        self.DG = self._derive_directed_graph()
        return self

    def set_source_sink_pairs(self, pairs):
        self.source_sink_pairs = pairs
         # Store mapping for quick lookup
        self.source_map = {s: i for i, (s, t) in enumerate(pairs)}
        self.sink_map = {t: i for i, (s, t) in enumerate(pairs)}
        return self

    def _derive_directed_graph(self):
        DG = nx.DiGraph()
        DG.add_nodes_from(self.G.nodes)
        for u, v, data in self.G.edges(data=True):
            capacity = data.get('capacity', 1)
            DG.add_edge(u, v, capacity=capacity)
            DG.add_edge(v, u, capacity=capacity)
        return DG

    def get_all_cuts(self):
        nodes = list(self.G.nodes)
        cuts = []
        # Limit combinations for very large graphs if needed
        max_nodes_for_all_cuts = 10 
        num_nodes = len(nodes)
        if num_nodes > max_nodes_for_all_cuts:
             print(f"Warning: Graph has {num_nodes} nodes. Calculating all cuts may be slow.")
             # Implement sampling or heuristics if needed for large graphs
        
        # Generate all non-empty proper subsets
        for i in range(1, 1 << num_nodes):
             S = set()
             for j in range(num_nodes):
                  if (i >> j) & 1:
                       S.add(nodes[j])
             if S and len(S) < num_nodes: # Ensure non-empty proper subset
                S_complement = set(nodes) - S
                cuts.append((S, S_complement))
        # Remove duplicate partitions (e.g., ({a}, {b,c}) and ({b,c}, {a}))
        unique_cuts = set()
        final_cuts = []
        for s_set, sc_set in cuts:
            s_tuple = tuple(sorted(list(s_set)))
            sc_tuple = tuple(sorted(list(sc_set)))
            # Canonical representation (lexicographically smaller first)
            cut_repr = tuple(sorted((s_tuple, sc_tuple)))
            if cut_repr not in unique_cuts:
                unique_cuts.add(cut_repr)
                final_cuts.append((s_set, sc_set))

        return final_cuts


    def get_incoming_edges(self, S):
        """Get incoming graph edges to a set of nodes S."""
        incoming = []
        for v in S:
            for u in self.DG.predecessors(v):
                if u not in S:
                    incoming.append((u, v))
        return incoming

    def get_outgoing_edges(self, S):
        """Get outgoing graph edges from a set of nodes S."""
        outgoing = []
        for u in S:
            for v in self.DG.successors(u):
                if v not in S:
                    outgoing.append((u, v))
        return outgoing

    def formulate_lp(self):
        """
        Formulate a linear program for network capacity analysis.
        """
        prob = LpProblem("Network_Capacity", LpMaximize)

        # 1. Create variables for all sessions (source-sink pairs)
        for i, (source, sink) in enumerate(self.source_sink_pairs):
            var_name = f"X{i+1}"
            self.entropy_vars[var_name] = LpVariable(var_name, lowBound=0)

        # 2. Create variables for all directed edges
        for u, v in self.DG.edges():
            var_name = f"H_{u}_{v}"
            self.entropy_vars[var_name] = LpVariable(var_name, lowBound=0)

        # 3. Create variables for auxiliary cut entropies
        all_cuts = self.get_all_cuts()
        for S, S_complement in all_cuts:
            s_label = '_'.join(sorted(S))
            # Check if any inputs (edges or sources) or outputs (edges or sinks) exist for S
            has_inputs = bool(self.get_incoming_edges(S) or any(s in S for s,t in self.source_sink_pairs))
            has_outputs = bool(self.get_outgoing_edges(S) or any(t in S for s,t in self.source_sink_pairs))

            if has_inputs:
                self.entropy_vars[f"H_in_{s_label}"] = LpVariable(f"H_in_{s_label}", lowBound=0)
            if has_outputs:
                 self.entropy_vars[f"H_out_{s_label}"] = LpVariable(f"H_out_{s_label}", lowBound=0)
            # Need H_in_out only if both inputs and outputs exist conceptually for IO inequality
            if has_inputs and has_outputs:
                 self.entropy_vars[f"H_in_out_{s_label}"] = LpVariable(f"H_in_out_{s_label}", lowBound=0)


        # 4. Set objective: maximize H(X1) (or average rate)
        prob += self.entropy_vars["X1"]

        # 5. Add constraint: all sessions have the same entropy rate R
        session_rate = self.entropy_vars["X1"] # R
        for i in range(1, len(self.source_sink_pairs)):
            prob += self.entropy_vars[f"X{i+1}"] == session_rate

        # 6. Add capacity constraints for undirected edges
        for u, v in self.G.edges():
            # Ensure variables exist before adding constraint
            h_uv_name = f"H_{u}_{v}"
            h_vu_name = f"H_{v}_{u}"
            if h_uv_name in self.entropy_vars and h_vu_name in self.entropy_vars:
                 prob += (self.entropy_vars[h_uv_name] + self.entropy_vars[h_vu_name]
                         <= self.G[u][v].get('capacity', 1)), f"Capacity_{u}_{v}"

        # 7. Add Crypto Inequalities for all cuts
        # H(Sessions_crossing_cut | Edges_crossing_cut) = 0
        # Relaxed to: sum H(Xi crossing) <= H_cut <= sum H(Edges crossing)
        # print("Adding Crypto Constraints...") # Added print statement
        crypto_constraints_added = 0
        for S, S_complement in all_cuts: # Assuming all_cuts is available
            s_label = '_'.join(sorted(S))

            # DEM(S): Sessions crossing the cut
            dem_S_vars = []
            dem_indices = [] # Store indices for variable names if needed
            for i, (s, t) in enumerate(self.source_sink_pairs):
                if (s in S and t in S_complement) or (s in S_complement and t in S):
                    var_name = f"X{i+1}"
                    if var_name in self.entropy_vars:
                        dem_S_vars.append(self.entropy_vars[var_name])
                        dem_indices.append(i+1)

            # CUT(S): Edges crossing the cut (in both directions)
            cut_S_edge_vars = []
            cut_edge_names = [] # Store names for debugging if needed
            crossing_edges = []
            # Edges S -> S_complement
            for u in S:
                for v in self.DG.successors(u):
                    if v in S_complement:
                        crossing_edges.append((u,v))
            # Edges S_complement -> S
            for u in S_complement:
                for v in self.DG.successors(u):
                    if v in S:
                        crossing_edges.append((u,v))

            for u, v in crossing_edges:
                edge_name = f"H_{u}_{v}"
                if edge_name in self.entropy_vars:
                    cut_S_edge_vars.append(self.entropy_vars[edge_name])
                    cut_edge_names.append(edge_name)

            # Only add if there are sessions demanding transfer across the cut
            # and edges allowing transfer across the cut
            if dem_S_vars and cut_S_edge_vars:
                # Create auxiliary variable H_cut_S
                cut_var_name = f"H_cut_{s_label}"
                # Avoid recreating if it already exists (can happen with symmetric cuts)
                if cut_var_name not in self.entropy_vars:
                    H_cut_S = LpVariable(cut_var_name, lowBound=0)
                    self.entropy_vars[cut_var_name] = H_cut_S
                else:
                    H_cut_S = self.entropy_vars[cut_var_name]
                
                crypto_constraints_added += 1

                # Upper bound H_cut_S by sum of crossing edge entropies
                prob += H_cut_S <= lpSum(cut_S_edge_vars), f"CryptoUpperBound_{s_label}"

                # Monotonicity: Each crossing edge entropy <= H_cut_S
                for edge_var in cut_S_edge_vars:
                    prob += edge_var <= H_cut_S, f"CryptoMonoEdge_{edge_var.name}_vs_{s_label}"

                # Crypto Inequality Relaxation: sum H(Sessions crossing) <= H_cut_S
                prob += lpSum(dem_S_vars) <= H_cut_S, f"Crypto_{s_label}"

        # Optional: Print how many were added
        print(f"Added {crypto_constraints_added} Crypto Inequality constraint sets.")



        # 7. Add input-output and related monotonicity constraints for all cuts
        for S, S_complement in all_cuts:
            s_label = '_'.join(sorted(S))
            
            # Identify components for this cut
            in_edges = self.get_incoming_edges(S)
            out_edges = self.get_outgoing_edges(S)
            
            sources_in_S_idx = [i for i, (s, t) in enumerate(self.source_sink_pairs) if s in S]
            sinks_in_S_idx = [i for i, (s, t) in enumerate(self.source_sink_pairs) if t in S]

            sources_in_S_vars = [self.entropy_vars[f"X{i+1}"] for i in sources_in_S_idx]
            sinks_in_S_vars = [self.entropy_vars[f"X{i+1}"] for i in sinks_in_S_idx] # Used for H_out and H_in_out bounds

            in_edge_vars = [self.entropy_vars[f"H_{u}_{v}"] for u, v in in_edges]
            out_edge_vars = [self.entropy_vars[f"H_{u}_{v}"] for u, v in out_edges]

            # Define H_in_S and its relations (if it exists)
            in_S_name = f"H_in_{s_label}"
            if in_S_name in self.entropy_vars:
                H_in_S = self.entropy_vars[in_S_name]
                # Upper bound relaxation: H_in_S <= sum of components
                prob += H_in_S <= lpSum(in_edge_vars) + lpSum(sources_in_S_vars), f"UpperBound_{in_S_name}"
                # Monotonicity: Each component <= H_in_S
                for edge_var in in_edge_vars:
                    prob += edge_var <= H_in_S, f"MonoInEdge_{edge_var.name}_in_{s_label}"
                for source_var in sources_in_S_vars:
                    prob += source_var <= H_in_S, f"MonoInSource_{source_var.name}_in_{s_label}"

            # Define H_out_S and its relations (if it exists)
            out_S_name = f"H_out_{s_label}"
            if out_S_name in self.entropy_vars:
                H_out_S = self.entropy_vars[out_S_name]
                 # Upper bound relaxation: H_out_S <= sum of components (edges out + sinks in S)
                prob += H_out_S <= lpSum(out_edge_vars) + lpSum(sinks_in_S_vars), f"UpperBound_{out_S_name}"
                 # Monotonicity: Each component <= H_out_S
                for edge_var in out_edge_vars:
                     prob += edge_var <= H_out_S, f"MonoOutEdge_{edge_var.name}_out_{s_label}"
                for sink_var in sinks_in_S_vars: # X_i where t_i in S
                     prob += sink_var <= H_out_S, f"MonoOutSink_{sink_var.name}_out_{s_label}"


            # Define H_in_out_S and apply IO (if it exists)
            both_S_name = f"H_in_out_{s_label}"
            if both_S_name in self.entropy_vars:
                H_in_out_S = self.entropy_vars[both_S_name]
                
                # Upper bound relaxation: H_in_out_S <= sum of all components (in/out edges + sources/sinks in S)
                prob += H_in_out_S <= lpSum(in_edge_vars) + lpSum(out_edge_vars) + \
                                      lpSum(sources_in_S_vars) + lpSum(sinks_in_S_vars), f"UpperBound_{both_S_name}"

                # Monotonicity H_in <= H_in_out and H_out <= H_in_out
                if in_S_name in self.entropy_vars:
                    prob += self.entropy_vars[in_S_name] <= H_in_out_S, f"Mono_In_vs_InOut_{s_label}"
                if out_S_name in self.entropy_vars:
                     prob += self.entropy_vars[out_S_name] <= H_in_out_S, f"Mono_Out_vs_InOut_{s_label}"

                # Apply Input-Output Inequality Relaxation: H(in U out) <= H(in)
                # Use the variables representing the upper bounds
                if in_S_name in self.entropy_vars: # Check H_in_S exists
                    prob += H_in_out_S <= self.entropy_vars[in_S_name], f"IO_{s_label}"


        # 8. Add submodularity constraints (Placeholder - complex to add all)
        # Example: H(A) + H(B) >= H(A U B) + H(A n B)
        # This is partially captured by the monotonicity constraints added above.
        # Adding all submodularity constraints is usually infeasible.

        # 9. Add joint decodability constraint per sink (Relaxation: sum H(Xi) <= sum H(in_edges))
        sink_to_sessions = defaultdict(list)
        for i, (source, sink) in enumerate(self.source_sink_pairs):
            var_name = f"X{i+1}"
            sink_to_sessions[sink].append(self.entropy_vars[var_name])

        for sink, session_vars_at_sink in sink_to_sessions.items():
            in_edges_to_sink = list(self.DG.in_edges(sink))
            if in_edges_to_sink: # Only if sink has incoming edges
                in_edge_vars_to_sink = [self.entropy_vars[f"H_{u}_{v}"] for u, v in in_edges_to_sink]
                # Constraint: Sum(H(Xi) for sink) <= Sum(H(Y_uv) for incoming edges)
                prob += lpSum(session_vars_at_sink) <= lpSum(in_edge_vars_to_sink), f"JointDecodability_{sink}"

        # 10. Relate Session variables to Cut entropy (Monotonicity: Xi <= H(Cut))
        for i, (source, sink) in enumerate(self.source_sink_pairs):
            session_var = self.entropy_vars[f"X{i+1}"]

            for S, S_complement in all_cuts:
                if ((source in S and sink in S_complement) or
                    (source in S_complement and sink in S)):
                    # If session (i) crosses cut S, then H(Xi) <= H(delta_in(S) U delta_out(S))
                    # Use H_in_out_S as the upper bound variable
                    s_label = '_'.join(sorted(S))
                    both_S_name = f"H_in_out_{s_label}"
                    if both_S_name in self.entropy_vars: # Check variable exists
                         prob += session_var <= self.entropy_vars[both_S_name], f"SessionCut_{session_var.name}_vs_{s_label}"

        return prob

    def analyze_network(self):
        """
        Analyze the network by formulating and solving the LP.
        """
        prob = self.formulate_lp()
        
        # Optional: Write LP to file for debugging
        # prob.writeLP("network_capacity.lp") 
        
        print("Solving LP...")
        # Solve with CBC solver, enable messages for debugging if needed
        prob.solve(PULP_CBC_CMD(msg=False)) # Set msg=True to see solver output
        
        print(f"Solver status: {LpStatus[prob.status]}")
        
        self.results['status'] = LpStatus[prob.status]
        if prob.status == LpStatusOptimal:
            self.results['max_rate'] = value(prob.objective)
            var_values = {}
            for name, var in self.entropy_vars.items():
                var_values[name] = var.value()
            self.results['variables'] = var_values
        elif prob.status == LpStatusInfeasible:
             print("Error: The LP problem is infeasible. Check constraints.")
             self.results['max_rate'] = None
        else:
             print(f"Warning: Optimal solution not found. Status: {LpStatus[prob.status]}")
             try:
                  self.results['max_rate'] = value(prob.objective) # May be None or incorrect
             except:
                  self.results['max_rate'] = None


        return self.results

    def print_results(self):
        """Print the analysis results."""
        print("\n====== Network Capacity Analysis Results ======")
        print(f"Network: {len(self.G.nodes)} nodes, {len(self.G.edges())} edges")
        print(f"Source-sink pairs: {self.source_sink_pairs}")
        print(f"LP Status: {self.results.get('status', 'N/A')}")


        if 'max_rate' in self.results and self.results['max_rate'] is not None:
            print(f"\nMaximum achievable rate (Upper Bound): {self.results['max_rate']:.8f}")

            if 'variables' in self.results:
                print("\nSession Variable Values:")
                for i in range(len(self.source_sink_pairs)):
                     var_name = f"X{i+1}"
                     val = self.results['variables'].get(var_name, 'N/A')
                     print(f"H({var_name}) = {val if val != 'N/A' else val:.8f}")

                # Optional: Print edge variables or other details if needed for debugging
                print("\nExample Edge Variable Values:")
                count = 0
                for name, value in self.results['variables'].items():
                    if name.startswith("H_") and "_" in name and len(name.split('_')) == 3 and count < 20:
                        print(f"{name} = {value:.4f}")
                        count += 1
                print("\nExample Cut Variable Values:")
                count = 0
                for name, value in self.results['variables'].items():
                     if name.startswith("H_in_out") and count < 10:
                          print(f"{name} = {value:.4f}")
                          count += 1

        else:
             print("\nCould not determine maximum achievable rate.")

        print("\n===========================================")


# Example Usage
if __name__ == "__main__":
    analyzer = EntropyCalculusAnalyzer()

    edges = [
        ('a', 'd'), ('a', 'e'),
        ('b', 'd'), ('b', 'e'),
        ('c', 'd'), ('c', 'e')
    ]
    analyzer.create_network(edges)

    # Use the ACYCLIC configuration from the paper's Theorem 1
    pairs = [
        ('a', 'b'),  # X1
        ('b', 'c'),  # X2
        ('a', 'c'),  # X3
        ('d', 'e')   # X4
    ]
    # # Optional: Use the CYCLIC configuration from Theorem 3
    # pairs = [
    #     ('a', 'b'),  # X1
    #     ('b', 'c'),  # X2
    #     ('c', 'a'),  # X3  <- Changed
    #     ('d', 'e')   # X4
    # ]
    analyzer.set_source_sink_pairs(pairs)

    analyzer.analyze_network()
    analyzer.print_results()

Added 14 Crypto Inequality constraint sets.
Solving LP...
Solver status: Optimal

====== Network Capacity Analysis Results ======
Network: 5 nodes, 6 edges
Source-sink pairs: [('a', 'b'), ('b', 'c'), ('a', 'c'), ('d', 'e')]
LP Status: Optimal

Maximum achievable rate (Upper Bound): 1.00000000

Session Variable Values:
H(X1) = 1.00000000
H(X2) = 1.00000000
H(X3) = 1.00000000
H(X4) = 1.00000000

Example Edge Variable Values:
H_a_d = 1.0000
H_a_e = 1.0000
H_d_a = 0.0000
H_d_b = 0.0000
H_d_c = 1.0000
H_e_a = 0.0000
H_e_b = 1.0000
H_e_c = 1.0000
H_b_d = 1.0000
H_b_e = 0.0000
H_c_d = 0.0000
H_c_e = 0.0000
H_in_a = 1.0000
H_out_a = 1.0000
H_in_d = 1.0000
H_out_d = 1.0000
H_in_e = 1.0000
H_out_e = 1.0000
H_in_b = 1.0000
H_out_b = 1.0000

Example Cut Variable Values:
H_in_out_a = 1.0000
H_in_out_d = 1.0000
H_in_out_a_d = 1.0000
H_in_out_e = 1.0000
H_in_out_a_e = 1.0000
H_in_out_d_e = 1.0000
H_in_out_a_d_e = 1.0000
H_in_out_b = 1.0000
H_in_out_a_b = 1.0000
H_in_out_b_d = 1.0000

